In [1]:
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.4/30.4 MB 75.5 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.2/920.2 kB 115.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 113.6 MB/s eta 0:00:00


In [2]:
import spacy
from spacy import displacy
from collections import Counter
import pandas as pd
pd.options.display.max_rows = 600
pd.options.display.max_colwidth = 400

In [3]:
!python -m spacy download pt_core_news_md

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 50.6 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 70.5 MB/s eta 0:00:0031m65.0 MB/s eta 0:00:01
  Attempting uninstall: spacy
    Found existing installation: spacy 3.8.0
    Uninstalling spacy-3.8.0:
      Successfully uninstalled spacy-3.8.0
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_md')


In [16]:
!pip install -U rel_pipe
!pip install -U rel_model

# make the factory work
from rel_pipe import make_relation_extractor, score_relations

# make the config work
from rel_model import create_relation_model, create_classification_layer, create_instances

import pt_core_news_md
nlp = pt_core_news_md.load()

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
ERROR: Could not find a version that satisfies the requirement rel_pipe (from versions: none)
ERROR: No matching distribution found for rel_pipe
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
ERROR: Could not find a version that satisfies the requirement rel_model (from versions: none)
ERROR: No matching distribution found for rel_model


ModuleNotFoundError: No module named 'rel_pipe'

In [7]:
filepath = '../files/texto01.txt'
text = open(filepath, encoding='utf-8').read()
document = nlp(text)

In [8]:
document.ents

(/4/ Senhora
 
 Senhora Rivail
 
 Residente,
 Boudet
 
 Propre no Château-du-Loir
 
 Sarthe,
 Carimbo,
 PARIS,
 Carimbo,
 CHÂTEAU DU LOIR,
 Queria,
 Amélie,
 Crispoul,
 Grandins,
 Leroux,
 Lesquillers,
 Ferran,
 Morin,
 Vorler,
 Éloïse,
 Aix,
 Éloïse,
 Éloïse,
 Lévi,
 Amélie,
 HLDRivail)

In [9]:
for named_entity in document.ents:
    print(named_entity, named_entity.label_)

/4/ Senhora

Senhora Rivail

Residente LOC
Boudet

Propre no Château-du-Loir

Sarthe PER
Carimbo MISC
PARIS ORG
Carimbo MISC
CHÂTEAU DU LOIR MISC
Queria ORG
Amélie PER
Crispoul PER
Grandins LOC
Leroux PER
Lesquillers MISC
Ferran PER
Morin PER
Vorler LOC
Éloïse PER
Aix LOC
Éloïse PER
Éloïse PER
Lévi PER
Amélie PER
HLDRivail PER


In [11]:
import math
number_of_chunks = 80

chunk_size = math.ceil(len(text) / number_of_chunks)

text_chunks = []

for number in range(0, len(text), chunk_size):
    text_chunk = text[number:number+chunk_size]
    text_chunks.append(text_chunk)

In [12]:
chunked_documents = list(nlp.pipe(text_chunks))

In [13]:
people = []

for document in chunked_documents:
    for named_entity in document.ents:
        if named_entity.label_ == "PER":
            people.append(named_entity.text)

people_tally = Counter(people)

df = pd.DataFrame(people_tally.most_common(), columns=['character', 'count'])
df

,character,count
0,Amélie,2
1,Éloïse,2
2,Boudet\n\nPropre no Château-du-Loir\n\nSarthe,1
3,/1/ Queria,1
4,Crispoul,1
5,Leroux,1
6,o.\n\n,1
7,Ferran,1
8,ecido,1
9,nder,1
